# M2177.003100 Deep Learning <br> Final Project: Text-to-Image Synthesis 

Copyright (C) Data Science & AI Laboratory, Seoul National University. This material is for educational uses only. Some contents are based on the material provided by other paper/book authors and may be copyrighted by them. 

**For understanding of this work, please carefully look at given PDF file.**

**Note**: certain details are missing or ambiguous on purpose, in order to test your knowledge on the related materials. However, if you really feel that something essential is missing and cannot proceed to the next step, then contact the teaching staff with clear description of your problem.

### Submitting your work:
<font color=red>**DO NOT clear the training process **</font> so that TAs can grade both your code and results.  


## 1. Load datasets

In [1]:
# pycharm 오류로 인해 직접 지정해줘야 돌아가서 넣었습니다
import os
os.chdir('/home/jaehyun/Downloads/22Final/')

In [2]:
import torch
from torch.autograd import Variable
import torchvision.transforms as transforms
import torch.optim as optim
from torch.utils.data._utils.collate import default_convert

from transformers import BertTokenizer, BertModel
import numpy as np

from config import cfg, cfg_from_file
import pprint
import datetime
import dateutil.tz
import time
import clip
from tensorboardX import SummaryWriter
from models import *
from encoders import CNN_ENCODER, RNN_ENCODER
from dataset import CUBDataset
from utils import *
from loss import *

2022-12-23 11:11:16.307113: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-23 11:11:16.389552: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-23 11:11:16.673437: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-11.3/lib64
2022-12-23 11:11:16.673473: W tensorflow/compiler/xla/stream_executor/

In [3]:
# Set a config file as 'train_birds.yml' in training, as 'eval_birds.yml' for evaluation
cfg_from_file('cfg/train_birds.yml') # eval_birds.yml

print('Using config:')
pprint.pprint(cfg)

os.environ['CUDA_VISIBLE_DEVICES'] = cfg.GPU_ID

now = datetime.datetime.now(dateutil.tz.tzlocal())
timestamp = now.strftime('%Y_%m_%d_%H_%M_%S')

os.makedirs(cfg.CHECKPOINT_DIR, exist_ok=True)

Using config:
{'BATCH_SIZE': 256,
 'CA': True,
 'CHECKPOINT_DIR': './checkpoints',
 'CHECKPOINT_NAME': '',
 'CNN': {'EMBEDDING_DIM': 32, 'H_DIM': 256},
 'CONFIG_NAME': 'text-to-image',
 'CUDA': True,
 'DATASET_NAME': 'birds',
 'DATA_DIR': 'data/birds',
 'EMBEDDING_TYPE': 'cnn-rnn',
 'GAN': {'B_ATTENTION': False,
         'B_CONDITION': False,
         'B_DCGAN': False,
         'CONDITION_DIM': 100,
         'DF_DIM': 32,
         'EMBEDDING_DIM': 256,
         'GF_DIM': 32,
         'R_NUM': 2,
         'Z_DIM': 100},
 'GPU_ID': '0, 1',
 'IMAGE_SIZE': 128,
 'NUM_BATCH_FOR_TEST': 4,
 'RANDOM_SEED': 1234,
 'RNN': {'EMBEDDING_DIM': 0,
         'H_DIM': 0,
         'TYPE': 'LSTM',
         'VOCAB_SIZE': 0,
         'WORD_EMBEDDING_DIM': 0},
 'R_PRECISION_DIR': '',
 'R_PRECISION_FILE': '',
 'R_PRECISION_FILE_HIDDEN': '',
 'TEST': {'B_EXAMPLE': False,
          'GENERATED_HIDDEN_TEST_IMAGES': '',
          'GENERATED_TEST_IMAGES': './evaluation/generated_images'},
 'TEXT': {'CAPTIONS_PER_IM

In [4]:
imsize = cfg.TREE.BASE_SIZE * (4 ** (cfg.TREE.BRANCH_NUM - 1))
image_transform = transforms.Compose([
    transforms.Resize(int(imsize)),
    transforms.RandomCrop(imsize),
    transforms.RandomHorizontalFlip()])

train_dataset = CUBDataset(cfg.DATA_DIR, transform=image_transform, split='train')

print(f'train data directory:\n{train_dataset.split_dir}')
print(f'# of train filenames:{train_dataset.filenames.shape}')
print(f'example of filename of train image:{train_dataset.filenames[0]}')
print(f'example of caption and its ids:\n{train_dataset.captions[0]}\n{train_dataset.captions_ids[0]}\n')
print(f'# of train captions:{np.asarray(train_dataset.captions).shape}')
print(f'# of train caption ids:{np.asarray(train_dataset.captions_ids).shape}')

self.current_dir:
/home/jaehyun/Downloads/22Final

self.data_dir:
/home/jaehyun/Downloads/22Final/data/birds

self.image_dir:
/home/jaehyun/Downloads/22Final/data/birds/CUB-200-2011/images

filepath /home/jaehyun/Downloads/22Final/data/birds/captions.pickle
Load from:  /home/jaehyun/Downloads/22Final/data/birds/captions.pickle
train data directory:
/home/jaehyun/Downloads/22Final/data/birds/train
# of train filenames:(8855,)
example of filename of train image:002.Laysan_Albatross/Laysan_Albatross_0002_1027
example of caption and its ids:
['a', 'bird', 'with', 'a', 'very', 'long', 'wing', 'span', 'and', 'a', 'long', 'pointed', 'beak']
[1, 2, 3, 1, 4, 5, 6, 7, 8, 1, 5, 9, 10]

# of train captions:(88550,)
# of train caption ids:(88550,)


/tmp/ipykernel_2600959/3361715005.py:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  print(f'# of train captions:{np.asarray(train_dataset.captions).shape}')
/tmp/ipykernel_2600959/3361715005.py:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  print(f'# of train caption ids:{np.asarray(train_dataset.captions_ids).shape}')


In [5]:
def my_collate_fn(samples):
    imgs = list()
    imgs.append(torch.stack([sample['img'][0] for sample in samples], dim=0))
    imgs.append(torch.stack([sample['img'][1] for sample in samples], dim=0))
    imgs.append(torch.stack([sample['img'][2] for sample in samples], dim=0))
    clip_enc = torch.stack([sample['clip_enc'] for sample in samples], dim=0)

    caps = torch.from_numpy(np.array([sample['caps'] for sample in samples]))
    cap_len = torch.from_numpy(np.array([sample['cap_len'] for sample in samples]))
    cls_id = torch.from_numpy(np.array([sample['cls_id'] for sample in samples]))
    key = [sample['key'] for sample in samples]
    sent_ix = torch.from_numpy(np.array([sample['sent_ix'] for sample in samples]))
    bert_enc = torch.stack([torch.tensor(sample['bert_enc']) for sample in samples], dim=0)
    bert_mask = torch.stack([torch.tensor(sample['bert_mask']) for sample in samples], dim=0)

    data = {'img': imgs, 'bert_enc': bert_enc, 'bert_mask': bert_mask, 'clip_enc': clip_enc, 'caps': caps, 'cap_len': cap_len, 'cls_id': cls_id, 'key': key, 'sent_ix': sent_ix}
    return data

In [6]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=cfg.BATCH_SIZE,
        drop_last=True, shuffle=True, num_workers=int(cfg.WORKERS), collate_fn=my_collate_fn)

## 2. Define models and optimizers

In [7]:
##### Build network
if cfg.TRAIN.RNN_ENCODER == '':
    raise ValueError('Error: no pretrained text-image encoders')

image_encoder = CNN_ENCODER(cfg.TEXT.EMBEDDING_DIM)
img_encoder_path = cfg.TRAIN.CNN_ENCODER
state_dict =  torch.load(img_encoder_path, map_location=lambda storage, loc: storage)
image_encoder.load_state_dict(state_dict)
for p in image_encoder.parameters():
    p.requires_grad = False
print('Load image encoder from:', img_encoder_path)
image_encoder.eval()

# baseline: RNN Enocder
text_encoder = RNN_ENCODER(train_dataset.n_words, nhidden=cfg.TEXT.EMBEDDING_DIM)
state_dict = torch.load(cfg.TRAIN.RNN_ENCODER, map_location=lambda storage, loc: storage)
text_encoder.load_state_dict(state_dict)
for p in text_encoder.parameters():
    p.requires_grad = False
print('Load text encoder from:', cfg.TRAIN.RNN_ENCODER)
text_encoder.eval()

# new model: BERT(mini)
#from transformers import AutoModel
#bert_encoder = AutoModel.from_pretrained("prajjwal1/bert-mini")
#for param in bert_encoder.parameters(): # freeze parameter for BERT
#    param.require_grad = False
#bert_encoder.eval()

if cfg.CUDA:
    text_encoder = text_encoder.cuda()
    image_encoder = image_encoder.cuda()
#    bert_encoder = bert_encoder.cuda()

Load image encoder from: ./evaluation/sim_models/bird/image_encoder.pth
Load text encoder from: ./evaluation/sim_models/bird/text_encoder.pth


In [8]:
#######################################################
# TODO
# (1) Build Generator and Discriminator Network
######################################################

generator = GENERATOR()
generator.apply(weights_init)

generator = torch.nn.DataParallel(generator)

discriminators = list()
if cfg.TREE.BRANCH_NUM > 0:
    discriminators.append(D_NET32())
if cfg.TREE.BRANCH_NUM > 1:
    discriminators.append(D_NET64())
if cfg.TREE.BRANCH_NUM > 2:
    discriminators.append(D_NET128())

for i in range(len(discriminators)):
    discriminators[i].apply(weights_init)
    discriminators[i] = torch.nn.DataParallel(discriminators[i])

if cfg.CUDA:
    generator.cuda()
    for i in range(len(discriminators)):
        discriminators[i].cuda()

##### Build Optimizers
#######################################################
# TODO
# (2) define optimizers for Generator and Discriminator
######################################################

optimizersD = list()
num_D = len(discriminators)
for i in range(num_D):
    opt = optim.Adam(
        discriminators[i].parameters(),
        lr=cfg.TRAIN.DISCRIMINATOR_LR,
        betas=(0.5, 0.999)
    )
    optimizersD.append(opt)

optimizerG = optim.Adam(
    generator.parameters(),
    lr=cfg.TRAIN.GENERATOR_LR,
    betas=(0.5, 0.999)
)

## 3. Set hyperparameters and go to train models

In [9]:
bs = cfg.BATCH_SIZE
nz = cfg.GAN.Z_DIM
noise = Variable(torch.FloatTensor(bs, nz))
fixed_noise = Variable(torch.FloatTensor(bs, nz).normal_(0, 1))
real_labels, fake_labels, match_labels = prepare_labels(bs)
if cfg.CUDA:
    noise, fixed_noise = noise.cuda(), fixed_noise.cuda()
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [10]:
gen_iterations = 0
summary_D = SummaryWriter() # tensorboard
summary_G = SummaryWriter()
summary_KL = SummaryWriter()
predictions = list()
clip_loss = CLIPLoss()

for epoch in range(cfg.TRAIN.MAX_EPOCH):
    start_t = time.time()

    data_iter = iter(train_dataloader)
    step = 0
    print('length of dataloader', len(train_dataloader))
    while step < len(train_dataloader):
        print('step:', step)
        data = next(data_iter)
        imgs, bert_enc, bert_mask, clip_enc, captions, cap_lens, class_ids, keys, sent_idx = prepare_data(data)
        #######################################################
        # TODO
        # (3) Compute Text Embeddings using RNN Encoder
        ######################################################
        #bert_embedding = bert_encoder(bert_enc)[1] # new model: BERT text embedding
        #hidden = (torch.zeros(2, cfg.BATCH_SIZE, 128).to(device=device), torch.zeros(2, cfg.BATCH_SIZE, 128).to(device=device)) # baseline
        hidden = text_encoder.init_hidden(captions.size(0))
        text_embedding = text_encoder(captions, cap_lens, hidden)

        #######################################################
        # TODO
        # (4) Generate fake images using Generator
        ######################################################
        noise.data.normal_(0, 1)
        fake_imgs, mu, logvar = generator(noise, text_embedding[1], z2w=False) # baseline
        #fake_imgs, mu, logvar = generator(noise, text_embedding, z2w=True) # new
        #fake_imgs, mu, logvar = generator(noise, bert_embedding, z2w=False)
        #######################################################
        # TODO
        # (5) classify real and fake images using Discriminator
        # (6) Update D network with adversarial loss
        ######################################################
        errD = 0
        for i in range(num_D):
            criterion = nn.BCELoss()
            discriminator, optimizer = discriminators[i], optimizersD[i]
            real_img = imgs[i]
            fake_img = fake_imgs[i]

            discriminator.zero_grad()
            real_logits = discriminator(real_img, mu.detach())
            fake_logits = discriminator(fake_img.detach(), mu.detach())

            # cond loss
            errD_real = criterion(real_logits[0], real_labels)
            errD_fake = criterion(fake_logits[0], fake_labels)

            # uncond loss
            errD_real_uncond = cfg.TRAIN.COEFF.UNCOND_LOSS * criterion(real_logits[1], real_labels)
            errD_fake_uncond = cfg.TRAIN.COEFF.UNCOND_LOSS * criterion(real_logits[1], real_labels)

            errD_real = errD_real + errD_real_uncond
            errD_fake = errD_fake + errD_fake_uncond

            errD = errD_real + errD_fake

            errD.backward()
            optimizer.step()


        #######################################################
        # TODO
        # (7) Update G network with adversarial loss
        ######################################################
        errG = 0
        generator.zero_grad()
        criterion = nn.BCELoss()

        for i in range(num_D):
            out = discriminators[i](fake_imgs[i], mu)
            errG = criterion(out[0], real_labels) # cond loss
            errG_patch = cfg.TRAIN.COEFF.UNCOND_LOSS * criterion(out[1], real_labels) # uncond loss
            errG += errG_patch

        #CLIP LOSS
        loss_clip = clip_loss(fake_imgs[-1], clip_enc.squeeze(1))
        errG += loss_clip

        #KL Loss
        kl_loss = KL_loss(mu, logvar) * cfg.TRAIN.COEFF.KL
        errG += kl_loss
        errG.backward()
        optimizerG.step()

        ####### logging training
        avg_param_G = deepcopy(list(p.data for p in generator.parameters()))
        for p, avg_p in zip(generator.parameters(), avg_param_G):
                    avg_p.mul_(0.999).add_(0.001, p.data)

        if step%100 == 0:
            summary_D.add_scalar('D_loss', errD.data)
            summary_G.add_scalar('G_loss', errG.data)
            summary_KL.add_scalar('KL_loss', kl_loss.data)

        step += 1

        gen_iterations += 1

    end_t = time.time()

    print('''[%d/%d][%d]
                    Loss_D: %.2f Loss_G: %.2f Time: %.2fs'''
            % (epoch+1, cfg.TRAIN.MAX_EPOCH, len(train_dataloader),
                errD.item(), errG.item(),
                end_t - start_t))

    if epoch % cfg.TRAIN.SNAPSHOT_INTERVAL == 0:  # and epoch != 0:
        save_model(generator, text_encoder, image_encoder, model_name='model_epoch_%03d.pth'%(epoch+1))
        #save_model(generator, bert_encoder, image_encoder, model_name='model_epoch_%03d.pth'%(epoch+1))

summary_D.close()
summary_G.close()
summary_KL.close()

save_model(generator, text_encoder, image_encoder)
#save_model(generator, bert_encoder, image_encoder)

length of dataloader 34
step: 0


/home/jaehyun/anaconda3/envs/geometry/lib/python3.9/site-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/tmp/ipykernel_2600959/340804107.py:95: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484809662/work/torch/csrc/utils/python_arg_parser.cpp:1174.)
  avg_p.mul_(0.999).add_(0.001, p.data)


step: 1
step: 2
step: 3
step: 4
step: 5
step: 6
step: 7
step: 8
step: 9
step: 10
step: 11
step: 12
step: 13
step: 14
step: 15
step: 16
step: 17
step: 18
step: 19
step: 20
step: 21
step: 22
step: 23
step: 24
step: 25
step: 26
step: 27
step: 28
step: 29
step: 30
step: 31
step: 32
step: 33
[1/400][34]
                    Loss_D: 1.28 Loss_G: 1.59 Time: 79.58s
length of dataloader 34
step: 0
step: 1
step: 2
step: 3
step: 4
step: 5
step: 6
step: 7
step: 8
step: 9
step: 10
step: 11
step: 12
step: 13
step: 14
step: 15
step: 16
step: 17
step: 18
step: 19
step: 20
step: 21
step: 22
step: 23
step: 24
step: 25
step: 26
step: 27
step: 28
step: 29
step: 30
step: 31
step: 32
step: 33
[2/400][34]
                    Loss_D: 1.16 Loss_G: 2.67 Time: 75.22s
length of dataloader 34
step: 0
step: 1
step: 2
step: 3
step: 4
step: 5
step: 6
step: 7
step: 8
step: 9
step: 10
step: 11
step: 12
step: 13
step: 14
step: 15
step: 16
step: 17
step: 18
step: 19
step: 20
step: 21
step: 22
step: 23
step: 24
step: 25
st

step: 1
step: 2
step: 3
step: 4
step: 5
step: 6
step: 7
step: 8
step: 9
step: 10
step: 11
step: 12
step: 13
step: 14
step: 15
step: 16
step: 17
step: 18
step: 19
step: 20
step: 21
step: 22
step: 23
step: 24
step: 25
step: 26
step: 27
step: 28
step: 29
step: 30
step: 31
step: 32
step: 33
[22/400][34]
                    Loss_D: 1.23 Loss_G: 2.23 Time: 74.54s
length of dataloader 34
step: 0
step: 1
step: 2
step: 3
step: 4
step: 5
step: 6
step: 7
step: 8
step: 9
step: 10
step: 11
step: 12
step: 13
step: 14
step: 15
step: 16
step: 17
step: 18
step: 19
step: 20
step: 21
step: 22
step: 23
step: 24
step: 25
step: 26
step: 27
step: 28
step: 29
step: 30
step: 31
step: 32
step: 33
[23/400][34]
                    Loss_D: 1.24 Loss_G: 1.65 Time: 74.49s
length of dataloader 34
step: 0
step: 1
step: 2
step: 3
step: 4
step: 5
step: 6
step: 7
step: 8
step: 9
step: 10
step: 11
step: 12
step: 13
step: 14
step: 15
step: 16
step: 17
step: 18
step: 19
step: 20
step: 21
step: 22
step: 23
step: 24
step: 25


length of dataloader 34
step: 0
step: 1
step: 2
step: 3
step: 4
step: 5
step: 6
step: 7
step: 8
step: 9
step: 10
step: 11
step: 12
step: 13
step: 14
step: 15
step: 16
step: 17
step: 18
step: 19
step: 20
step: 21
step: 22
step: 23
step: 24
step: 25
step: 26
step: 27
step: 28
step: 29
step: 30
step: 31
step: 32
step: 33
[43/400][34]
                    Loss_D: 0.96 Loss_G: 1.69 Time: 75.20s
length of dataloader 34
step: 0
step: 1
step: 2
step: 3
step: 4
step: 5
step: 6
step: 7
step: 8
step: 9
step: 10
step: 11
step: 12
step: 13
step: 14
step: 15
step: 16
step: 17
step: 18
step: 19
step: 20
step: 21
step: 22
step: 23
step: 24
step: 25
step: 26
step: 27
step: 28
step: 29
step: 30
step: 31
step: 32
step: 33
[44/400][34]
                    Loss_D: 0.66 Loss_G: 2.53 Time: 75.25s
length of dataloader 34
step: 0
step: 1
step: 2
step: 3
step: 4
step: 5
step: 6
step: 7
step: 8
step: 9
step: 10
step: 11
step: 12
step: 13
step: 14
step: 15
step: 16
step: 17
step: 18
step: 19
step: 20
step: 21
step

...

length of dataloader 34
step: 0
step: 1
step: 2
step: 3
step: 4
step: 5
step: 6
step: 7
step: 8
step: 9
step: 10
step: 11
step: 12
step: 13
step: 14
step: 15
step: 16
step: 17
step: 18
step: 19
step: 20
step: 21
step: 22
step: 23
step: 24
step: 25
step: 26
step: 27
step: 28
step: 29
step: 30
step: 31
step: 32
step: 33
[316/400][34]
                    Loss_D: 0.12 Loss_G: 4.46 Time: 75.26s
length of dataloader 34
step: 0
step: 1
step: 2
step: 3
step: 4
step: 5
step: 6
step: 7
step: 8
step: 9
step: 10
step: 11
step: 12
step: 13
step: 14
step: 15
step: 16
step: 17
step: 18
step: 19
step: 20
step: 21
step: 22
step: 23
step: 24
step: 25
step: 26
step: 27
step: 28
step: 29
step: 30
step: 31
step: 32
step: 33
[317/400][34]
                    Loss_D: 0.17 Loss_G: 3.13 Time: 75.18s
length of dataloader 34
step: 0
step: 1
step: 2
step: 3
step: 4
step: 5
step: 6
step: 7
step: 8
step: 9
step: 10
step: 11
step: 12
step: 13
step: 14
step: 15
step: 16
step: 17
step: 18
step: 19
step: 20
step: 21
st

length of dataloader 34
step: 0
step: 1
step: 2
step: 3
step: 4
step: 5
step: 6
step: 7
step: 8
step: 9
step: 10
step: 11
step: 12
step: 13
step: 14
step: 15
step: 16
step: 17
step: 18
step: 19
step: 20
step: 21
step: 22
step: 23
step: 24
step: 25
step: 26
step: 27
step: 28
step: 29
step: 30
step: 31
step: 32
step: 33
[337/400][34]
                    Loss_D: 0.02 Loss_G: 7.72 Time: 75.36s
length of dataloader 34
step: 0
step: 1
step: 2
step: 3
step: 4
step: 5
step: 6
step: 7
step: 8
step: 9
step: 10
step: 11
step: 12
step: 13
step: 14
step: 15
step: 16
step: 17
step: 18
step: 19
step: 20
step: 21
step: 22
step: 23
step: 24
step: 25
step: 26
step: 27
step: 28
step: 29
step: 30
step: 31
step: 32
step: 33
[338/400][34]
                    Loss_D: 0.61 Loss_G: 2.29 Time: 75.31s
length of dataloader 34
step: 0
step: 1
step: 2
step: 3
step: 4
step: 5
step: 6
step: 7
step: 8
step: 9
step: 10
step: 11
step: 12
step: 13
step: 14
step: 15
step: 16
step: 17
step: 18
step: 19
step: 20
step: 21
st

length of dataloader 34
step: 0
step: 1
step: 2
step: 3
step: 4
step: 5
step: 6
step: 7
step: 8
step: 9
step: 10
step: 11
step: 12
step: 13
step: 14
step: 15
step: 16
step: 17
step: 18
step: 19
step: 20
step: 21
step: 22
step: 23
step: 24
step: 25
step: 26
step: 27
step: 28
step: 29
step: 30
step: 31
step: 32
step: 33
[358/400][34]
                    Loss_D: 0.05 Loss_G: 5.54 Time: 75.25s
length of dataloader 34
step: 0
step: 1
step: 2
step: 3
step: 4
step: 5
step: 6
step: 7
step: 8
step: 9
step: 10
step: 11
step: 12
step: 13
step: 14
step: 15
step: 16
step: 17
step: 18
step: 19
step: 20
step: 21
step: 22
step: 23
step: 24
step: 25
step: 26
step: 27
step: 28
step: 29
step: 30
step: 31
step: 32
step: 33
[359/400][34]
                    Loss_D: 0.08 Loss_G: 6.41 Time: 75.36s
length of dataloader 34
step: 0
step: 1
step: 2
step: 3
step: 4
step: 5
step: 6
step: 7
step: 8
step: 9
step: 10
step: 11
step: 12
step: 13
step: 14
step: 15
step: 16
step: 17
step: 18
step: 19
step: 20
step: 21
st

length of dataloader 34
step: 0
step: 1
step: 2
step: 3
step: 4
step: 5
step: 6
step: 7
step: 8
step: 9
step: 10
step: 11
step: 12
step: 13
step: 14
step: 15
step: 16
step: 17
step: 18
step: 19
step: 20
step: 21
step: 22
step: 23
step: 24
step: 25
step: 26
step: 27
step: 28
step: 29
step: 30
step: 31
step: 32
step: 33
[379/400][34]
                    Loss_D: 0.07 Loss_G: 5.47 Time: 75.35s
length of dataloader 34
step: 0
step: 1
step: 2
step: 3
step: 4
step: 5
step: 6
step: 7
step: 8
step: 9
step: 10
step: 11
step: 12
step: 13
step: 14
step: 15
step: 16
step: 17
step: 18
step: 19
step: 20
step: 21
step: 22
step: 23
step: 24
step: 25
step: 26
step: 27
step: 28
step: 29
step: 30
step: 31
step: 32
step: 33
[380/400][34]
                    Loss_D: 0.06 Loss_G: 6.88 Time: 75.32s
length of dataloader 34
step: 0
step: 1
step: 2
step: 3
step: 4
step: 5
step: 6
step: 7
step: 8
step: 9
step: 10
step: 11
step: 12
step: 13
step: 14
step: 15
step: 16
step: 17
step: 18
step: 19
step: 20
step: 21
st

length of dataloader 34
step: 0
step: 1
step: 2
step: 3
step: 4
step: 5
step: 6
step: 7
step: 8
step: 9
step: 10
step: 11
step: 12
step: 13
step: 14
step: 15
step: 16
step: 17
step: 18
step: 19
step: 20
step: 21
step: 22
step: 23
step: 24
step: 25
step: 26
step: 27
step: 28
step: 29
step: 30
step: 31
step: 32
step: 33
[400/400][34]
                    Loss_D: 0.01 Loss_G: 8.81 Time: 81.49s
